In [2]:
import torch
import transformers
import zarr

In [3]:
enc = torch.hub.load("facebookresearch/dinov2", "dinov2_vitl14").eval().cuda()

enc.num_features

Using cache found in /home/larsankile/.cache/torch/hub/facebookresearch_dinov2_main
/home/larsankile/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/larsankile/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/larsankile/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


1024

In [4]:
# Check how many out features the model has
print(enc.num_features)

1024


In [12]:
trans = transformers.AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = transformers.AutoModel.from_pretrained("facebook/dinov2-base").eval().cuda()

In [13]:
model.config.hidden_size

768

In [6]:
d = zarr.open(
    "/home/larsankile/furniture-diffusion/data/processed/sim/image/low/one_leg/data.zarr"
)

In [7]:
img = d["image1"][:100]
img.shape

(100, 224, 224, 3)

In [10]:
trans(torch.Tensor(img), return_tensors="pt").pixel_values.shape

torch.Size([100, 3, 224, 224])

In [25]:
model(trans(img, return_tensors="pt").pixel_values.cuda()).mean()

OutOfMemoryError: CUDA out of memory. Tried to allocate 102.00 MiB (GPU 0; 10.76 GiB total capacity; 9.11 GiB already allocated; 64.94 MiB free; 9.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
with torch.no_grad():
    res = model(
        trans(img, return_tensors="pt").pixel_values.cuda()
    ).pooler_output.isclose(
        enc(trans(img, return_tensors="pt").pixel_values.cuda()), atol=1e-4
    )

res.all()

tensor(True, device='cuda:0')

In [14]:
model

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-23): 24 x Dinov2Layer(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2Attention(
          (attention): Dinov2SelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_